In [1]:
import numpy as np
import pylab as plt
import planet_search as ps

In [3]:
# just do it
tic = 98796344 # ltt 1445
#tic = 318022259 # toi-1730
#tic = 307210830  # l 98-59
tics = [101955023,288314668,16530308,289972535,402313808]  # Kristo'S FAP calculation

for i in range(1,len(tics)):
    ts = ps.run_full_planet_search(tics[i])

  0%|          | 0/2544 periods | 00:00<?

Running TLS for planet 1 (sector(s) 14)
Transit Least Squares TLS 1.0.28 (13 June 2021)
Creating model cache for 40 durations
Searching 18380 data points, 2544 periods from 0.601 to 16.985 days
Using all 8 CPU threads


100%|██████████| 2544/2544 periods | 00:26<00:00
  3%|▎         | 258/9989 [00:00<00:07, 1301.66it/s]

Searching for best T0 for period 2.28246 days


100%|██████████| 9989/9989 [00:03<00:00, 3134.28it/s]
  0%|          | 0/2544 periods | 00:00<?

...

100%|██████████| 2284/2284 periods | 00:26<00:00
  0%|          | 0/2284 periods | 00:00<?

Running TLS for planet 2 (sector(s) 29)
Transit Least Squares TLS 1.0.28 (13 June 2021)
Creating model cache for 39 durations
Searching 14096 data points, 2284 periods from 0.601 to 15.969 days
Using all 8 CPU threads


100%|██████████| 2284/2284 periods | 00:23<00:00
  0%|          | 0/2284 periods | 00:00<?

Running TLS for planet 3 (sector(s) 29)
Transit Least Squares TLS 1.0.28 (13 June 2021)
Creating model cache for 39 durations
Searching 13393 data points, 2284 periods from 0.601 to 15.969 days
Using all 8 CPU threads


100%|██████████| 2284/2284 periods | 00:22<00:00
